<a href="https://colab.research.google.com/github/shwets1503/E-commerce_data_analysis_prediction/blob/master/Customer_Lifetime_Value_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To calculate Lifetime Value first we need to select a time window. It can be anything like 3, 6, 12, 24 months. By the equation below, we can have Lifetime Value for each customer in that specific time window:<br>
Lifetime Value: Total Gross Revenue - Total Cost<br>

This equation now gives us the historical lifetime value. If we see some customers having very high negative lifetime value historically, it could be too late to take an action. At this point, we need to predict the future with machine learning. We are going to build a simple machine learning model that predicts our customers lifetime value.

In order to calculate the value:

*  Define an appropriate time frame for Customer Lifetime Value calculation
* Identify the features we are going to use to predict future and create them
* Calculate lifetime value (LTV) for training the machine learning model
*Build and run the machine learning model
* Check if the model is useful




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# import libraries
from datetime import datetime, timedelta, date
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division

import plotly.graph_objects as go

from sklearn.metrics import classification_report,confusion_matrix
from __future__ import division
from sklearn.cluster import KMeans

import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Importing datasets
customer = pd.read_csv('../content/drive/My Drive/sales_data/customers_dataset.csv', engine='python')
orders_df = pd.read_csv('../content/drive/My Drive/sales_data2/olist_orders_dataset.csv', engine='python')
order_items = pd.read_csv('../content/drive/My Drive/sales_data2/olist_order_items_dataset.csv', engine='python')

In [4]:
orders = pd.merge(customer, orders_df, how ='left', on='customer_id')

In [5]:
orders = pd.merge(orders, order_items,how ='left', on='order_id')

In [6]:
col = ['customer_unique_id', 'price', 'order_item_id', 'order_purchase_timestamp']
orders = orders[col]

In [7]:
#converting the type of Invoice Date Field from string to datetime.
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

In [8]:
pd.to_datetime('2017-1-1')

Timestamp('2017-01-01 00:00:00')

In [9]:
orders.head()

,customer_unique_id,price,order_item_id,order_purchase_timestamp
0,861eff4711a542e4b93843c6dd7febb0,124.99,1.0,2017-05-16 15:05:35
1,290c77bc529b7ac935b93aa66c333dc3,289.00,1.0,2018-01-12 20:48:24
2,060e732b5b29e8181a18229c7b0b2b5e,139.94,1.0,2018-05-19 16:07:45
3,259dac757896d24d7702b9acbbff3f3c,149.94,1.0,2018-03-13 16:06:38
4,345ecd01c38d18a9036ed96c73b8d066,230.00,1.0,2018-07-29 09:51:30


In [10]:
#create 3m and 6m dataframes
orders_3m = orders[(orders.order_purchase_timestamp < pd.to_datetime('2017-6-1')) & (orders.order_purchase_timestamp >= pd.to_datetime('2017-3-1'))].reset_index(drop=True)
orders_6m = orders[(orders.order_purchase_timestamp >= pd.to_datetime('2017-6-1')) & (orders.order_purchase_timestamp  < pd.to_datetime('2017-12-1'))].reset_index(drop=True)

#create tx_user for assigning clustering
orders_user = pd.DataFrame(orders_3m['customer_unique_id'].unique())
orders_user.columns = ['customer_unique_id']

In [11]:
#order cluster method
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [12]:
#calculate recency score
orders_max_purchase = orders_3m.groupby('customer_unique_id').order_purchase_timestamp.max().reset_index()
orders_max_purchase.columns = ['customer_unique_id','MaxPurchaseDate']
orders_max_purchase['Recency'] = (orders_max_purchase['MaxPurchaseDate'].max() - orders_max_purchase['MaxPurchaseDate']).dt.days
orders_user = pd.merge(orders_user, orders_max_purchase[['customer_unique_id','Recency']], on='customer_unique_id')

kmeans = KMeans(n_clusters=4)
kmeans.fit(orders_user[['Recency']])
orders_user['RecencyCluster'] = kmeans.predict(orders_user[['Recency']])

orders_user = order_cluster('RecencyCluster', 'Recency',orders_user,False)

In [13]:
#calcuate frequency score
orders_frequency = orders_3m.groupby('customer_unique_id').order_purchase_timestamp.count().reset_index()
orders_frequency.columns = ['customer_unique_id','Frequency']
orders_user = pd.merge(orders_user, orders_frequency, on='customer_unique_id')

kmeans = KMeans(n_clusters=4)
kmeans.fit(orders_user[['Frequency']])
orders_user['FrequencyCluster'] = kmeans.predict(orders_user[['Frequency']])

orders_user = order_cluster('FrequencyCluster', 'Frequency',orders_user,True)

In [14]:
#calcuate revenue score
orders_3m['Revenue'] = orders_3m['price'] * orders_3m['order_item_id']
orders_revenue = orders_3m.groupby('customer_unique_id').Revenue.sum().reset_index()
orders_user = pd.merge(orders_user, orders_revenue, on='customer_unique_id')

kmeans = KMeans(n_clusters=4)
kmeans.fit(orders_user[['Revenue']])
orders_user['RevenueCluster'] = kmeans.predict(orders_user[['Revenue']])
orders_user = order_cluster('RevenueCluster', 'Revenue',orders_user,True)

In [15]:
#overall scoring
orders_user['OverallScore'] = orders_user['RecencyCluster'] + orders_user['FrequencyCluster'] + orders_user['RevenueCluster']
orders_user['Segment'] = 'Low-Value'
orders_user.loc[orders_user['OverallScore']>2,'Segment'] = 'Mid-Value' 
orders_user.loc[orders_user['OverallScore']>4,'Segment'] = 'High-Value' 

In [16]:
orders_user.head()

,customer_unique_id,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment
0,861eff4711a542e4b93843c6dd7febb0,15,3,1,0,124.99,0,3,Mid-Value
1,7f3a72e8f988c6e735ba118d54f47458,20,3,1,0,89.90,0,3,Mid-Value
2,2e6a42a9b5cbb0da62988694f18ee295,16,3,1,0,29.99,0,3,Mid-Value
3,fd2d5fdb84e65fa6b54b98b0e2df5645,9,3,1,0,44.90,0,3,Mid-Value
4,8728c766c84eeda24b3e54fe6e632051,1,3,1,0,117.30,0,3,Mid-Value


In [17]:
#calculate revenue and create a new dataframe for it
orders_6m['Revenue'] = orders_6m['price'] * orders_6m['order_item_id']
orders_user_6m = orders_6m.groupby('customer_unique_id')['Revenue'].sum().reset_index()
orders_user_6m.columns = ['customer_unique_id','m6_Revenue']


#plot LTV histogram
plot_data = [
    go.Histogram(
        x=orders_user_6m.query('m6_Revenue < 10000')['m6_Revenue']
    )
]

plot_layout = go.Layout(
        title='6m Revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

In [18]:
orders_merge = pd.merge(orders_user, orders_user_6m, on='customer_unique_id', how='left')
orders_merge = orders_merge.fillna(0)

orders_graph = orders_merge.query("m6_Revenue < 30000")

plot_data = [
    go.Scatter(
        x=orders_graph.query("Segment == 'Low-Value'")['OverallScore'],
        y=orders_graph.query("Segment == 'Low-Value'")['m6_Revenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=orders_graph.query("Segment == 'Mid-Value'")['OverallScore'],
        y=orders_graph.query("Segment == 'Mid-Value'")['m6_Revenue'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=orders_graph.query("Segment == 'High-Value'")['OverallScore'],
        y=orders_graph.query("Segment == 'High-Value'")['m6_Revenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "6m LTV"},
        xaxis= {'title': "RFM Score"},
        title='LTV'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

In [19]:
#remove outliers
orders_merge = orders_merge[orders_merge['m6_Revenue']<orders_merge['m6_Revenue'].quantile(0.99)]


#creating 3 clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(orders_merge[['m6_Revenue']])
orders_merge['LTVCluster'] = kmeans.predict(orders_merge[['m6_Revenue']])

#order cluster number based on LTV
orders_merge = order_cluster('LTVCluster', 'm6_Revenue',orders_merge,True)

#creatinga new cluster dataframe
orders_cluster = orders_merge.copy()

#see details of the clusters
orders_cluster.groupby('LTVCluster')['m6_Revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
LTVCluster,,,,,,,,
0,8452.0,0.007508,0.295046,0.00,0.0000,0.0,0.00,13.99
1,36.0,32.378611,11.175493,16.30,20.7475,34.9,39.99,49.95
2,27.0,72.509259,11.796669,56.99,59.9000,72.0,82.60,93.00


In [20]:
#convert categorical columns to numerical
orders_class = pd.get_dummies(orders_cluster)

In [21]:
#calculate and show correlations
corr_matrix = orders_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

LTVCluster                                             1.000000
m6_Revenue                                             0.975310
customer_unique_id_a0e0770d529c5da6dd71d5be8620aa3b    0.166346
customer_unique_id_35de25c73544f00e8adde226a4d1bdcc    0.166346
customer_unique_id_82fb46292a999e7db9e2391c407f76e4    0.166346
                                                         ...   
customer_unique_id_7f3a72e8f988c6e735ba118d54f47458   -0.000884
Revenue                                               -0.003356
RevenueCluster                                        -0.010598
Recency                                               -0.022013
Segment_Low-Value                                     -0.024765
Name: LTVCluster, Length: 8527, dtype: float64

In [22]:
#create X and y, X will be feature set and y is the label - LTV
X = orders_class.drop(['LTVCluster','m6_Revenue'],axis=1)
y = orders_class['LTVCluster']

In [23]:
#split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=56)

In [24]:
#XGBoost Multiclassification Model
ltv_xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1,objective= 'multi:softprob',n_jobs=-1).fit(X_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(ltv_xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(ltv_xgb_model.score(X_test[X_train.columns], y_test)))

y_pred = ltv_xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

Accuracy of XGB classifier on training set: 0.99
Accuracy of XGB classifier on test set: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       422
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         1

    accuracy                           0.99       426
   macro avg       0.33      0.33      0.33       426
weighted avg       0.98      0.99      0.99       426



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



I am getting 99% accuracy score on both train and test set which is odd. I need to investigate, future course of action is:

* Adding more features and improve feature engineering
* Try different models other than XGBoost
* Apply hyper parameter tuning to current model
* Add more data to the model if possible